In [2]:
import cupy as cp
import cusignal 
from scipy import signal
import numpy as np

### Resample

In [2]:
start = 0
stop = 10
num = int(1e8)
resample_num = int(1e5)

cx = np.linspace(start, stop, num, endpoint=False) 
cy = np.cos(-cx**2/6.0)

In [3]:
%%timeit
cf = signal.resample(cy, resample_num, window=('kaiser', 0.5))

14.1 s ± 272 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
gx = cp.linspace(start, stop, num, endpoint=False)
gy = cp.cos(-gx**2/6.0)

In [5]:
%%timeit
gf = cusignal.resample(gy, resample_num, window=('kaiser',0.5))

71.7 ms ± 4.04 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Resample Poly

In [6]:
start = 0
stop = 10
num = int(1e8)
resample_up = 2
resample_down = 3

cx = np.linspace(start, stop, num, endpoint=False) 
cy = np.cos(-cx**2/6.0)

In [7]:
%%timeit
cf = signal.resample_poly(cy, resample_up, resample_down, window=('kaiser', 0.5))

2.4 s ± 1.21 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
gx = cp.linspace(start, stop, num, endpoint=False)
gy = cp.cos(-gx**2/6.0)

In [9]:
%%timeit
cf = cusignal.resample_poly(gy, resample_up, resample_down, window=('kaiser', 0.5), use_numba=True)

32.4 ms ± 101 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
%%timeit
cf = cusignal.resample_poly(gy, resample_up, resample_down, window=('kaiser', 0.5), use_numba=False)

17.3 ms ± 26.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### FIR Filter Design with Window

In [11]:
numtaps = int(1e8)
f1, f2 = 0.1, 0.2

In [12]:
%%timeit
cfirwin = signal.firwin(numtaps, [f1, f2], pass_zero=False)

13.5 s ± 28.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
%%timeit
gfirwin = cusignal.firwin(numtaps, [f1, f2], pass_zero=False)

149 ms ± 230 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Correlate

In [14]:
sig = np.random.rand(int(1e8))
sig_noise = sig + np.random.randn(len(sig))

In [15]:
%%timeit
ccorr = signal.correlate(sig_noise, np.ones(128), mode='same') / 1e6

3.39 s ± 1.81 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
sig = cp.random.rand(int(1e8))
sig_noise = sig + cp.random.randn(len(sig))

In [17]:
%%timeit
gcorr = cusignal.correlate(sig_noise, cp.ones(128), mode='same') / 1e6

72.9 ms ± 30.8 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Convolve

In [18]:
sig = np.random.rand(int(1e8))
win = signal.windows.hann(int(1e3))

In [19]:
%%timeit
cconv = signal.convolve(sig, win, mode='same') / np.sum(sig)

16.7 s ± 1.11 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
sig = cp.random.rand(int(1e8))
win = cusignal.hann(int(1e3))

In [21]:
%%timeit
gconv = cusignal.convolve(sig, win, mode='same') / cp.sum(win)

73.1 ms ± 10.8 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Convolution using the FFT Method

In [22]:
csig = np.random.randn(int(1e8))

In [23]:
%%timeit
cautocorr = signal.fftconvolve(csig, csig[::-1], mode='full')

27.7 s ± 420 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [24]:
gsig = cp.random.randn(int(1e8))

In [25]:
%%timeit
gautocorr = cusignal.fftconvolve(gsig, gsig[::-1], mode='full')

128 ms ± 487 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Wiener Filter on N-Dimensional Array

In [26]:
csig = np.random.rand(int(1e8))

In [27]:
%%timeit
cfilt = signal.wiener(csig)

3.59 s ± 8.47 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [28]:
gsig = cp.random.randn(int(1e8))

In [29]:
%%timeit
gfilt = cusignal.wiener(gsig)

255 ms ± 42.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Perform 1-D Hilbert Transform

In [30]:
csig = np.random.rand((int(1e8)))

In [31]:
%%timeit
chtrans = signal.hilbert(csig)

11.1 s ± 63.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [32]:
gsig = cp.random.rand(int(1e8))

In [33]:
%%timeit
ghtrans = cusignal.hilbert(gsig)

71.2 ms ± 1.24 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Perform 2-D Hilbert Transform

In [34]:
csig = np.random.rand(int(1e4), int(1e4))

In [35]:
%%timeit
chtrans2d = signal.hilbert2(csig)

6.28 s ± 9.93 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [36]:
gsig = cp.random.rand(int(1e4), int(1e4))

In [37]:
%%timeit
ghtrans2d = cusignal.hilbert2(gsig)

86 ms ± 2.39 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Perform 2-D Convolution and Correlation

In [38]:
csig = np.random.rand(int(1e4), int(1e4))
filt = np.random.rand(5,5)

In [39]:
%%timeit
grad = signal.convolve2d(csig, filt, boundary='symm', mode='same')

8.23 s ± 5.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [40]:
%%timeit
grad = signal.correlate2d(csig, filt, boundary='symm', mode='same')

8.17 s ± 20.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [41]:
gsig = cp.random.rand(int(1e4), int(1e4))
gfilt = cp.random.rand(5,5)

In [42]:
%%timeit
ggrad = cusignal.convolve2d(gsig, gfilt, boundary='symm', mode='same', use_numba=True)

45.4 ms ± 381 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [43]:
%%timeit
ggrad = cusignal.convolve2d(gsig, gfilt, boundary='symm', mode='same', use_numba=False)

46.6 ms ± 686 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [44]:
%%timeit
ggrad = cusignal.correlate2d(gsig, gfilt, boundary='symm', mode='same', use_numba=True)

26 ms ± 250 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [45]:
%%timeit
ggrad = cusignal.correlate2d(gsig, gfilt, boundary='symm', mode='same', use_numba=False)

51.7 ms ± 608 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
